# Power Laws: Detecting Anomalies in Usage
Energy consumption of buildings has steadily increased. There is an increasing realization that many buildings do not perform as intended by their designers. Typical buildings consume 20% more energy than necessary due to faulty construction, malfunctioning equipment, incorrectly configured control systems and inappropriate operating procedures.

The building systems may fail to meet the performance expectations due to various faults. Poorly maintained, degraded, and improperly controlled equipment wastes an estimated 15% to 30% of energy used in commercial buildings.

Therefore, it is of great potential to develop automatic, quick-responding, accurate and reliable fault detection and to provide diagnosis schemes to ensure the optimal operations of systems to save energy.

Schneider Electric already has relevant offers, but would like to determine if alternative techniques can add new detections / functionalities, bring gain in precision, or operate with less data.

https://search.library.northwestern.edu/primo-explore/fulldisplay?docid=01NWU_HATHI_TRUSTMIU01-100664356&context=L&vid=NULVNEW&search_scope=NWU&tab=default_tab&lang=en_US

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import power_utils

%matplotlib inline

In [ ]:
preproc = power_utils.create_train_set()

D:\Users\Luis\Programs\Anaconda3\envs\PREDICT-env\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
